# Optimize the query plan II

Suppose we want to join questions with users (using the table from the metastore). We also want to use a UDF (which does some computation on the question message field) and using a window we want for each user to order the questions he/she asked depending on the creation date. 

See the query bellow which does that in suboptimal way and try to rewrite it to achieve more optimal plan. More specifically try to eliminate the Exchange operator in the query plan.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, udf, row_number
)

from pyspark.sql import Window
from pyspark.sql.types import IntegerType

import os

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('Optimize II')
    .enableHiveSupport()
    .getOrCreate()
)

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

questions_input_path = os.path.join(project_path, 'data/questions')

In [ ]:
# We will turn broadcast join off because we want to work with sort merge join (SMJ) because we want to assume that
# in practice both datasets are large so SMJ would manifest anyway

spark.conf.set('spark.sql.autoBroadcastJoinThreshold', -1)

In [ ]:
usersDF = spark.table('users')

questionsDF = (
    spark
    .read
    .option('path', questions_input_path)
    .load()    
)

#### UDF:

The UDF bellow is just simple function that gets the lenght of a string. This can be easily done using native pyspark dataframe function length. For the sake of this example however suppose that this function encapsulates some complex logic which can not be done natively.

In [ ]:
@udf(IntegerType())
def get_length_udf(str):
    return len(str)

#### Window definition:

In [ ]:
w = Window().partitionBy('user_id').orderBy('creation_date')

# Task:

The query bellow is suboptimal. Try to rewrite the query to achive more optimal plan that leads to more efficient execution.

Hint:
* see the query plan
* eliminate the Exchange from the plan

In [ ]:
(
    usersDF
    .join(questionsDF, 'user_id')
    .withColumn('question_len', get_length_udf('body'))
    .withColumn('question_n', row_number().over(w))
    .write
    .mode('overwrite')
    .format('noop')
    .save()
)

#### Rewrite the query:

Hint:
* move the UDF before the join - this will preserve your partitioning and will eliminate the suffle for the window function

In [ ]:
# your code here:



In [ ]:
spark.stop()